In [1]:
import sys
sys.setrecursionlimit(300000)

In [2]:
inp_sample = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

In [3]:
inp_sample_2 = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

In [4]:
inp = """%cf -> tz
%kr -> xn, gq
%cp -> sq, bd
broadcaster -> vn, sj, tg, kn
%hc -> pm
%fd -> xn, mj
%qz -> xf
%vf -> mc, pm
%zm -> rz, pm
%cn -> bd, qz
%jj -> bp
%ks -> ff
%nb -> xn, ks
%bm -> pm, vf
&xn -> kc, jb, cb, tg, ks, tx
%lm -> rk
%dn -> bd, cn
%ft -> dn
%pn -> pm, ll
%rk -> bp, fs
%tz -> bp, gp
%mc -> jx
%fs -> kx
%jf -> bd, fm
%rz -> hc, pm
%tg -> cb, xn
&hf -> rx
%vp -> pn
&pm -> ll, mc, sj, vd, vp
%rn -> kc, xn
%vn -> bd, cp
&nd -> hf
%fm -> bd, gc
%ff -> xn, fd
&bp -> cf, fh, pc, kn, fs, gn, lm
&pc -> hf
%mj -> xn
%qg -> bd
%fh -> lm
%kc -> nb
%xf -> bd, jf
%gc -> qg, bd
&bd -> vn, sq, qz, ft, nd
%jb -> kr
%gp -> bp, rp
%gq -> xn, rn
%sj -> pm, bm
%rp -> bp, jj
%sq -> ft
%cb -> jb
&vd -> hf
%gn -> cf
%kx -> gn, bp
%ll -> zm
&tx -> hf
%jx -> md, pm
%md -> pm, vp
%kn -> fh, bp"""

In [42]:
def solve1(inp):
    modules_list = [s.split(" -> ") for s in inp.splitlines()]
    modules = {}
    start = None
    for m in modules_list:
        targets = m[1].split(", ")
        if m[0] == "broadcaster":
            start = targets
        else:
            mtype = m[0][0]
            mname = m[0][1:]
            modules[mname] = {"type": mtype, "targets": targets}
            if mtype == "%":
                modules[mname]["ffon"] = False
            else:
                modules[mname]["input"] = {}
    
    for m in modules:
        for target in modules[m]["targets"]:
            if target in modules and modules[target]["type"] == "&":
                modules[target]["input"][m] = "low"
    for target in start:
        if target in modules and modules[target]["type"] == "&":
            modules[target]["input"]["broadcaster"] = "low"
    # print(modules)
    
    counts = {"low": 1000, "high": 0}
    
    for _ in range(1000):
        frontier = [("broadcaster", s, "low") for s in start]
        while len(frontier) > 0:
            next_frontier = []
            for source, mod, pulse in frontier:
                counts[pulse] += 1
                if mod not in modules:
                    continue
                mtype = modules[mod]["type"]
                targets = modules[mod]["targets"]

                if mtype == "%":
                    if pulse == "low":
                        ffon = modules[mod]["ffon"]
                        for target in targets:
                            if ffon:
                                next_frontier.append((mod, target, "low"))
                            else:
                                next_frontier.append((mod, target, "high"))
                        modules[mod]["ffon"] = not ffon
                else:
                    modules[mod]["input"][source] = pulse
                    all_high = True
                    for inp_pulse in modules[mod]["input"].values():
                        if inp_pulse != "high":
                            all_high = False
                            break
                    for target in targets:
                        if all_high:
                            next_frontier.append((mod, target, "low"))
                        else:
                            next_frontier.append((mod, target, "high"))
            frontier = next_frontier
    print(counts["low"])
    print(counts["high"])
    return counts["low"] * counts["high"]

In [43]:
solve1(inp_sample)

8000
4000


32000000

In [44]:
solve1(inp_sample_2)

4250
2750


11687500

In [45]:
solve1(inp)

17730
45520


807069600

In [73]:
def solve2(inp):
    modules_list = [s.split(" -> ") for s in inp.splitlines()]
    modules = {}
    start = None
    for m in modules_list:
        targets = m[1].split(", ")
        if m[0] == "broadcaster":
            start = targets
        else:
            mtype = m[0][0]
            mname = m[0][1:]
            modules[mname] = {"type": mtype, "targets": targets}
            if mtype == "%":
                modules[mname]["ffon"] = False
            else:
                modules[mname]["input"] = {}
    
    for m in modules:
        for target in modules[m]["targets"]:
            if target in modules and modules[target]["type"] == "&":
                modules[target]["input"][m] = "low"
    for target in start:
        if target in modules and modules[target]["type"] == "&":
            modules[target]["input"]["broadcaster"] = "low"
    
    ans = 1
    for bp in range(5000):
        frontier = [("broadcaster", s, "low") for s in start]
        while len(frontier) > 0:
            next_frontier = []
            for source, mod, pulse in frontier:
                if mod in ("tx", "pc", "nd", "vd") and pulse == "low":
                    ans *= (bp + 1)
                if mod not in modules:
                    continue
                mtype = modules[mod]["type"]
                targets = modules[mod]["targets"]

                if mtype == "%":
                    if pulse == "low":
                        ffon = modules[mod]["ffon"]
                        for target in targets:
                            if ffon:
                                next_frontier.append((mod, target, "low"))
                            else:
                                next_frontier.append((mod, target, "high"))
                        modules[mod]["ffon"] = not ffon
                else:
                    modules[mod]["input"][source] = pulse
                    all_high = True
                    for inp_pulse in modules[mod]["input"].values():
                        if inp_pulse != "high":
                            all_high = False
                            break
                    for target in targets:
                        if all_high:
                            next_frontier.append((mod, target, "low"))
                        else:
                            next_frontier.append((mod, target, "high"))
            frontier = next_frontier
    return ans

In [74]:
%%time
solve2(inp)

CPU times: user 134 ms, sys: 2.93 ms, total: 137 ms
Wall time: 137 ms


221453937522197